In [1]:
!pip install flask
!pip install pyserial
!pip install opencv-python
!pip install nbimporter
!pip install torch torchaudio torchvision
!jupyter nbconvert --to script text_processing.ipynb
!jupyter nbconvert --to script camera_operation.ipynb
!jupyter nbconvert --to script braille_translate.ipynb

[NbConvertApp] Converting notebook text_processing.ipynb to script
[NbConvertApp] Writing 7344 bytes to text_processing.py
[NbConvertApp] Converting notebook camera_operation.ipynb to script
[NbConvertApp] Writing 675 bytes to camera_operation.py
[NbConvertApp] Converting notebook braille_translate.ipynb to script
[NbConvertApp] Writing 6570 bytes to braille_translate.py


In [2]:
from flask import Flask
import serial
import time
import os
import cv2
import threading
import subprocess
import nbimporter
from camera_operation import taking_picture
from image_processing import crop_image
from text_processing import text_correction, text_detection
from braille_translate import text_to_braille

app = Flask(__name__)

# 연결할 시리얼 포트와 baudrate 설정
ser = serial.Serial('COM7', 9600)

# 변수 선언 및 초기화
camera = cv2.VideoCapture(2)
recognized_texts_global = []
current_text_index = 0
braille_texts = []

def read_from_port(ser):
    # 인식된 텍스트를 저장할 전역 변수
    global recognized_texts_global
    global current_text_index
    global braille_texts

    while True:
        reading = ser.readline().decode().strip()  # 끝의 줄바꿈 제거
        print("===============  Received from Arduino -> ", reading, "  ===============")
        if reading == "Take Picture":
            # 이미지 촬영
            print("< 이미지 촬영 >")
            img_path = taking_picture(camera)

            print("< 텍스트 인식 >")
            recognized_texts = text_detection(img_path)
            # 인식된 텍스트 처리
            words_list = [word for text in recognized_texts for word in text.split()]
            corrected_texts = text_correction(words_list)
            filtered_texts = [text for text in corrected_texts if text.strip()]
            
            print("인식된 텍스트 : ", filtered_texts)
           # recognized_texts_global = filtered_texts  # 새로운 텍스트로 업데이트
           # current_text_index = 0  # 새로운 텍스트 리스트로 업데이트되면 인덱스 초기화

            
            # 텍스트를 점자로 변환하여 저장
            print("< 텍스트 점자로 변환 >")
            braille_texts = text_to_braille(filtered_texts)
            print("점자 텍스트 : ", braille_texts)
            recognized_texts_global = braille_texts
            current_text_index = 0
        elif reading == "Next":
            next_text()
            
        elif reading == "Previous":
            previous_text()
            
def next_text():
    global current_text_index
    
    if len(recognized_texts_global) == 0:
        print("리스트가 비어있습니다.")
        
    elif current_text_index < len(recognized_texts_global) - 1:
        send_text_to_arduino(recognized_texts_global[current_text_index])
        current_text_index += 1
    else:
        print("리스트의 끝점입니다.")

def previous_text():
    global current_text_index
    
    if len(recognized_texts_global) == 0:
        print("리스트가 비어있습니다.")
        
    elif current_text_index > 0:
        send_text_to_arduino(recognized_texts_global[current_text_index])
        current_text_index -= 1
        
    else:
        print("리스트의 시작점입니다.")

def send_text_to_arduino(text):
    print("Sending to Arduino:", text)
    ser.write((text + "\n").encode())

# 프로그램 종료 시 자원 해제
def release_resources():
    camera.release()
    ser.close()

@app.route('/')
def index():
    return "Serial Communication with Arduino"

thread = threading.Thread(target=read_from_port, args=(ser,))
thread.start()

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5000)

KeyboardInterrupt: 